# <h2> Step 3 – script 4 - Aggregate statistics across conditions
<br>
    <br>1. Total number of events 
    <br>2. Percentage events (# events / total events)
    <br> Output - Output: step3_4_aggregate_stats.csv

In [1]:
import os
import numpy as np
import scipy.io as sio
import pandas as pd
from __future__ import division

In [3]:
# Enter root directory
root_dir = "/Path/to/EMBODY/Files"

# Enter analyses
analysis = "phase1_demo"

# Enter step2 subj list
subjects = [124]
group_step2 = ["control"]

In [4]:
def read_txt_file(filename):
    """Read meditation file, returns array of digits

        filename: input is string (e.g., "step2_3cat.txt")

    """

    with open(filename, 'rw') as f:
        content = f.readlines()
    content = list(content[0])

    # filter items in content by whether or not they are a digit. if digit, convert to integer.
    content = [int(x) for x in content if x.isdigit()] 

    return content

In [5]:
# Compute aggregate stats
def aggregate_stats(): 
    
    events_count=[0,0,0,0,0]
    
    for index in range(len(meditation)):
        current_value = meditation[index]
        previous_value = meditation[index-1]
        if index == 0 and current_value != 0:        
            events_count[current_value-1] += 1
        elif current_value == 0:
            continue
        elif current_value != previous_value:
            events_count[current_value-1] += 1
    
    total = sum(events_count)
    
    breath_count = events_count[0] # index 0 = breath
    stop_count = events_count[2] # index 2 = stop
    self_count = events_count[3] # index 3 = self
    
    breath_event_pct = (breath_count/total)*100
    stop_event_pct = (stop_count/total)*100
    self_event_pct = (self_count/total)*100
    
    aggStats = [total, breath_event_pct, stop_event_pct, self_event_pct]
    
    return aggStats

In [6]:
# Convert aggregate stats to dataframe and write data
def compile_stats():
    
    agg_stats = aggregate_stats()
    
    index_list = ['event_total', 'event_pct_breath', 'event_pct_stop', 'event_pct_self']   
    
    df = pd.DataFrame(agg_stats, index = index_list, columns = [subj]) # convert to pandas df
    
    df_ts = pd.DataFrame.transpose(df) # transpose to 1 row
    df_ts.to_csv('step3_4_3cat_aggregateStats.csv') # writes out data
    
    return None

In [8]:
### 1. Computes aggregate meditation stats in each condition for each subject

for subj in subjects:
    os.chdir('%s/%s/results/step2/%d/' % (root_dir, analysis, subj))
    infile = "step2_event_clean.txt"

    meditation = read_txt_file(infile)

    # write out data
    compile_stats()

In [9]:
## 2. Compile step3 stats across subjects - for SPSS
aggregate_stats_compile = []
    
appended_data = []

for subj in subjects:

    os.chdir('%s/%s/results/step2/%d/' % (root_dir, analysis, subj))
    data = pd.read_csv('step3_4_3cat_aggregateStats.csv', sep=',', index_col=0)

    # store DataFrame in list
    appended_data.append(data)

# Concatenate meditation stats
df = pd.concat(appended_data, axis=0) # concat by rows
df.insert(0, "group", group_step2) # insert group in first column after index

# write DataFrame to csv 
os.chdir('%s/%s/results/step3_compile/' % (root_dir, analysis))
df.to_csv('step3_4_3cat_aggregate_stats.csv')